In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# cycle_data 폴더에 있는 모든 파일 불러옴
path = 'cycle_data/'
# 파일은 뒤에 숫자만 다름 '0,1,2,5,7,9,10,11,12,13,15,17,19,20,23,25,27,29,31,33'
files = [f'{path}cycle_{i}.csv' for i in [0, 1, 2, 5, 7, 9, 10, 11, 12, 13, 15, 17, 19, 20, 23, 25, 27, 29, 31, 33]]
files

['cycle_data/cycle_0.csv',
 'cycle_data/cycle_1.csv',
 'cycle_data/cycle_2.csv',
 'cycle_data/cycle_5.csv',
 'cycle_data/cycle_7.csv',
 'cycle_data/cycle_9.csv',
 'cycle_data/cycle_10.csv',
 'cycle_data/cycle_11.csv',
 'cycle_data/cycle_12.csv',
 'cycle_data/cycle_13.csv',
 'cycle_data/cycle_15.csv',
 'cycle_data/cycle_17.csv',
 'cycle_data/cycle_19.csv',
 'cycle_data/cycle_20.csv',
 'cycle_data/cycle_23.csv',
 'cycle_data/cycle_25.csv',
 'cycle_data/cycle_27.csv',
 'cycle_data/cycle_29.csv',
 'cycle_data/cycle_31.csv',
 'cycle_data/cycle_33.csv']

In [95]:
# 각 파일마다 효율성 구해서 데이터 저장

cycle_time = 1 / 2.5 # 1초에 2.5개 생산
cycle0_output = 13770.0 # cycle_0의 생산량
cycle10_output = 16952.5 # cycle_10의 생산량
cycle33_output = 29160.0 # cycle_33의 생산량

results = []

for file in files:
    df = pd.read_csv(file)
    
    df["Timestamp"] = pd.to_datetime(df["Timestamp"]) # Timestamp를 datetime으로 변환
    df["time_diff"] = df["Timestamp"].diff() # 시간차 계산

    ####################################### 시간 가동율
    # 가동시간
    df_status2 = df[df["STATUS"] == 2].copy() # STATUS == 2인 데이터 필터링
    run_time = df_status2.loc[df_status2["time_diff"] < pd.Timedelta(seconds=30), "time_diff"].dropna().dt.total_seconds().sum() # 30초 이상은 제거
    # 부하시간
    load_time = df.loc[df["time_diff"] < pd.Timedelta(seconds=30), "time_diff"].dropna().dt.total_seconds().sum() # 30초 이상은 제거

    oee1 = run_time / load_time
    

    ######################################## 성능 가동율
    work_time = run_time # 일일가동시간

    # oee2 계산에서 만약 cycle_0.csv 파일이라면 cycle0_output을 사용하고, ...
    # 나머지 파일은 day1_output을 사용
    if file == 'cycle_data/cycle_0.csv':
        day1_output = cycle0_output
    elif file == 'cycle_data/cycle_10.csv':
        day1_output = cycle10_output
    elif file == 'cycle_data/cycle_31.csv':
        work_time = 27261
    elif file == 'cycle_data/cycle_33.csv':
        day1_output = cycle33_output
        work_time = 12134
    else:
        day1_output = 65534.0 # 일일생산량

    oee2 = cycle_time * day1_output / work_time

    ######################################## 종합수율
    # 일단 1로 가정
    oee3 = 1.0

    ######################################## OEE
    oee = oee1 * oee2 * oee3

    # 결과 리스트에 저장
    results.append({
        "file": file,
        "OEE1": oee1,
        "OEE2": oee2,
        "OEE3": oee3,
        "OEE": oee
    })

# DataFrame으로 변환
df_results = pd.DataFrame(results)
print(df_results)
# 저장
df_results.to_csv("oee_results.csv", index=False)

                       file      OEE1      OEE2  OEE3       OEE
0    cycle_data/cycle_0.csv  0.938431  0.961089   1.0  0.901916
1    cycle_data/cycle_1.csv  0.612942  0.960944   1.0  0.589003
2    cycle_data/cycle_2.csv  0.842068  0.961367   1.0  0.809536
3    cycle_data/cycle_5.csv  0.871170  0.961438   1.0  0.837575
4    cycle_data/cycle_7.csv  0.874174  0.961790   1.0  0.840772
5    cycle_data/cycle_9.csv  0.883682  0.961402   1.0  0.849574
6   cycle_data/cycle_10.csv  0.603631  0.962117   1.0  0.580764
7   cycle_data/cycle_11.csv  0.995545  0.961438   1.0  0.957155
8   cycle_data/cycle_12.csv  0.877006  0.961367   1.0  0.843125
9   cycle_data/cycle_13.csv  0.800905  0.961826   1.0  0.770331
10  cycle_data/cycle_15.csv  0.875024  0.961473   1.0  0.841312
11  cycle_data/cycle_17.csv  0.922621  0.961297   1.0  0.886913
12  cycle_data/cycle_19.csv  0.819108  0.961473   1.0  0.787550
13  cycle_data/cycle_20.csv  0.828820  0.961297   1.0  0.796742
14  cycle_data/cycle_23.csv  0.667254  0

---

oee2의 값이 1 이상인 경우 ; 0, 31, 33 데이터 직접 확인

In [78]:
df0 = pd.read_csv('cycle_data/cycle_33.csv')

In [79]:
df0["Timestamp"] = pd.to_datetime(df0["Timestamp"]) # Timestamp를 datetime으로 변환
df0["time_diff"] = df0["Timestamp"].diff() # 시간차 계산

# 가동시간
df_status2 = df0[df0["STATUS"] == 2].copy() # STATUS == 2인 데이터 필터링
run_time0 = df_status2.loc[df_status2["time_diff"] < pd.Timedelta(seconds=30), "time_diff"].dropna().dt.total_seconds().sum() # 30초 이상은 제거
# 부하시간
load_time0 = df0.loc[df0["time_diff"] < pd.Timedelta(seconds=30), "time_diff"].dropna().dt.total_seconds().sum() # 30초 이상은 제거

oee1_0 = run_time0 / load_time0

In [84]:
df0[df0['STATUS'] != 2.0]

,Timestamp,OUTPUT_COUNT_DAY_1,KO6_MOTOR_SET_FREQ,CUTTING_SET_FREQ,STATUS,KO5_MOTOR_SET_FREQ,METAL_OIL_SUPPLY_PRESS_CONTR,KO4_MOTOR_SET_FREQ,KO2_MOTOR_SET_FREQ,MAIN_MOTOR_CURR,...,KO1_MOTOR_INVERTER_ALM,TRANS_POS_DOWN_SET_H,OUTPUT_COUNT_DAY_2,OUTPUT_COUNT_SUM,TRANS_POS_DOWN,TRANS_POS_RIGHT,MAIN_MOTOR_ALM,DAY_1_DIFF,CYCLE,time_diff
2090,2022-05-14 02:22:30,12930.5,6000.0,6000.0,0.0,6000.0,248.448,6000.0,6000.0,4101.25,...,1.0,4000.0,12930.5,1717.0,636.214,1731.00,0.0,4.0,33,0 days 00:00:02
2091,2022-05-14 02:22:31,12930.5,6000.0,6000.0,0.0,6000.0,257.894,6000.0,6000.0,3976.33,...,1.0,4000.0,12930.5,1717.0,636.214,1731.48,0.0,0.0,33,0 days 00:00:01
2092,2022-05-14 02:22:32,12930.5,6000.0,6000.0,0.0,6000.0,251.880,6000.0,6000.0,3900.00,...,1.0,4000.0,12930.5,1717.0,636.214,1731.50,0.0,0.0,33,0 days 00:00:01
2093,2022-05-14 02:22:33,12930.5,6000.0,6000.0,0.0,6000.0,251.000,6000.0,6000.0,3931.00,...,1.0,4000.0,12930.5,1717.0,636.214,1731.50,0.0,0.0,33,0 days 00:00:01
2094,2022-05-14 02:22:34,12930.5,6000.0,6000.0,0.0,6000.0,257.166,6000.0,6000.0,3966.00,...,1.0,4000.0,12930.5,1717.0,636.214,1731.60,0.0,0.0,33,0 days 00:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,2022-05-14 04:34:32,29160.0,6000.0,6000.0,0.0,6000.0,254.875,6000.0,6000.0,3905.00,...,1.0,4000.0,29160.0,1879.0,2319.000,2322.50,0.0,0.0,33,0 days 00:00:01
5877,2022-05-14 04:34:33,29160.0,6000.0,6000.0,0.0,6000.0,254.875,6000.0,6000.0,3905.00,...,1.0,4000.0,29160.0,1879.0,2319.000,2322.50,0.0,0.0,33,0 days 00:00:01
5878,2022-05-14 04:34:39,29160.0,6000.0,6000.0,0.0,6000.0,254.875,6000.0,6000.0,3905.00,...,1.0,4000.0,29160.0,1879.0,2319.000,2322.50,0.0,0.0,33,0 days 00:00:06
5879,2022-05-14 04:34:40,29160.0,6000.0,6000.0,0.0,6000.0,262.579,6000.0,6000.0,3900.42,...,1.0,4000.0,29160.0,1879.0,2319.000,2322.50,0.0,0.0,33,0 days 00:00:01


In [87]:
oee1_0, run_time0, load_time0

(np.float64(0.9566587864460205), np.float64(9712.0), np.float64(10152.0))